# Testing the implementation

In [1]:
import os
import gzip
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import tensorflow_datasets as tfds

from models import *
from trainer import *
from preprocess import *
from callbacks import *

/Users/customer/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 0 Data

In [2]:
parent_path = str(pathlib.Path(os.getcwd()).parent)
df = pd.read_csv(os.path.join(parent_path, 'data/data.csv'))
df = df.drop(['Unnamed: 0'], axis=1)
df.sample(5)

,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility
18812,150044566.0,506534.0,4.00,C,2006-01-19,0.3850,0.38200,4.77470,6.0,148.0,1.193675,0.047639,0.368776
11330,150374157.0,506534.0,5.10,C,2007-01-25,0.0705,0.07150,5.03030,435.0,50.0,0.986333,0.053997,0.105159
11560,155057994.0,701057.0,450.00,C,2006-12-28,0.2000,0.25000,414.01000,11.0,78.0,0.920022,0.054046,0.129193
9701,150256898.0,506534.0,4.65,C,2006-11-06,0.3005,0.31875,4.94060,13.0,39.0,1.062495,0.054198,0.087583
47091,153775435.0,702263.0,11.60,C,2013-03-04,0.2700,0.29875,11.99125,200.0,74.0,1.033728,0.002703,0.129568


Create the first neueral network that uses the same set of inputs as the *Black-Scholes model*

In [3]:
dataframe_BS = np.vstack((df['strike'].values,
                      df['underlyings_price'].values,
                      df['days_to_maturity'].values,
                      df['volatility'].values,
                      df['rate'].values,
                      df['contract_price'].values)).T                

In [4]:
train_ds, valid_ds, test_ds = pipeline1(dataframe_BS, scaling=False)

train_copy, valid_copy, test_copy = pipeline1(dataframe_BS, prefetch=False)

(85999, 6) (10750, 6) (10750, 6)
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

(85999, 6) (10750, 6) (10750, 6)


2022-09-01 19:40:19.781599: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-01 19:40:19.781754: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## 1 Start training

Define some parameters

In [5]:
print_num_epochs = 5 # print progress every print_num_epochs epochs

path_to_save = os.path.join(parent_path, 'NeuralNetwork/models/')  # path to save the model

patience = 10  

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

loss = tf.keras.losses.MeanAbsoluteError(name='loss')

metrics = tf.keras.metrics.MeanAbsolutePercentageError(name='accuracy')

num_epochs = 10  

input_shape = (5,)

num_layers = 3

hidden_units = [14, 14, 14]

output_shape = (1, )

batchnorm = True  

dropout = None

model = getModel(input_shape = input_shape,
                num_layers   = num_layers,
                 hidden_units = hidden_units,
                 output_shape = output_shape,
                 batchnorm = batchnorm,
                 dropout = dropout)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5)]               0         
                                                                 
 flatten (Flatten)           (None, 5)                 0         
                                                                 
 dense (Dense)               (None, 14)                84        
                                                                 
 batch_normalization (BatchN  (None, 14)               56        
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 14)                210       
                                                                 
 batch_normalization_1 (Batc  (None, 14)               56        
 hNormalization)                                             

Define some callbacks  

In [6]:
ckpt = CheckpointCallback(path_to_save)
printing =PrintProgress(num_epochs=print_num_epochs)
early_stop = tf.keras.callbacks.EarlyStopping(patience=patience, monitor='val_loss')
callbacks = [ckpt, printing, early_stop]

In [ ]:
history = compile_and_fit(model,
                          optimizer,
                          loss,
                          num_epochs,
                          train_ds,
                          valid_ds,
                          metrics,
                          callbacks
                          )

# 1.1 Tuning the hyperparameters

To tune the hyperparameter, we can use `keras_tuner`, which provides `RandomSearch`, `Hyperband`, and `BayesianSearch` methods for searching the best hyperparameters.  

To get started, we only tune the number of layers and the hidden units in each layer first, then we tune with learning rate together.

In [7]:
import keras_tuner

We initialize the `keras_tuner` object for tuning hyperparameters; currently, there are two hyper models implemented, `tuneLayer` and `tuneLR`

In [8]:
random_tuner = keras_tuner.RandomSearch(
    hypermodel=tunedModel, # the hypermodel to tune # can be tuneLR or tuneLayer
    objective="val_loss", # the objective to optimize
    max_trials=3, # the maximum number of trials to run
    executions_per_trial=2, # the number of models generated on each trial
    overwrite=True, # whether to overwrite previous trials
    directory="hyperparams/RandomSearch", # the directory to save the trials
    project_name="1", # the name of the project
)  

# get a summary of the range of hyperparameters to tune
random_tuner.search_space_summary()

Search space summary
Default search space size: 7
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
activation (Choice)
{'default': 'elu', 'conditions': [], 'values': ['elu', 'tanh'], 'ordered': False}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
rate_decay (Float)
{'default': 0.85, 'conditions': [], 'min_value': 0.85, 'max_value': 0.9995, 'step': None, 'sampling': None}
l1_coeff (Float)
{'default': 1e-08, 'conditions': [], 'min_value': 1e-08, 'max_value': 3.162277660168379e-07, 'step': None, 'sampling': None}
l2_coeff (Float)
{'default': 1e-08, 'conditions': [], 'min_value': 1e-08, 'max_value': 3.162277660168379e-07, 'step': None, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 7, 'step': 1, 'sampling': None}


In [9]:
# start the search
random_tuner.search(train_ds, epochs = 2, validation_data = valid_ds)    

Trial 3 Complete [00h 01m 13s]
val_loss: 44.001516342163086

Best val_loss So Far: 39.983163833618164
Total elapsed time: 00h 04m 27s
INFO:tensorflow:Oracle triggered exit


Get the best compiled models

In [10]:
models = random_tuner.get_best_models(num_models=1)
best_model = models[0]

best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5)]               0         
                                                                 
 flatten (Flatten)           (None, 5)                 0         
                                                                 
 dense (Dense)               (None, 6)                 36        
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 6)                 42        
                                                                 
 dense_3 (Dense)             (None, 5)                 35        
                                                                 
 dense_4 (Dense)             (None, 1)                 6     

We can view the results of the search and use the hyperparameters displayed to train the model.

In [11]:
random_tuner.results_summary()

Results summary
Results in hyperparams/RandomSearch/1
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 4
activation: elu
learning_rate: 0.0016288864664974797
rate_decay: 0.9173374660739638
l1_coeff: 1.42192594929355e-07
l2_coeff: 3.659583114428574e-08
units_1: 6
units_2: 6
units_3: 6
units_4: 5
Score: 39.983163833618164
Trial summary
Hyperparameters:
num_layers: 3
activation: tanh
learning_rate: 0.001023588238883239
rate_decay: 0.9343507716282039
l1_coeff: 1.0050528182507988e-07
l2_coeff: 4.591369951915932e-08
units_1: 6
units_2: 7
units_3: 7
units_4: 5
Score: 44.001516342163086
Trial summary
Hyperparameters:
num_layers: 3
activation: tanh
learning_rate: 0.0008458403820081236
rate_decay: 0.8727872979789447
l1_coeff: 1.4061275790324222e-07
l2_coeff: 2.6722175775868732e-08
units_1: 5
units_2: 5
units_3: 5
Score: 44.31102752685547


Now train the best model obtained

In [12]:
best_model.fit(train_ds, epochs = 5, validation_data = valid_ds)

Epoch 1/5
   6/2688 [..............................] - ETA: 29s - loss: 38.4511 - mean_squared_error: 0.0099  

2022-09-01 19:47:48.983083: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2686/2688 [============================>.] - ETA: 0s - loss: 36.7604 - mean_squared_error: 0.0084

2022-09-01 19:48:17.878414: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2688/2688 [==============================] - 31s 11ms/step - loss: 36.7566 - mean_squared_error: 0.0084 - val_loss: 32.2790 - val_mean_squared_error: 0.0069
Epoch 2/5
2688/2688 [==============================] - 31s 11ms/step - loss: 32.7288 - mean_squared_error: 0.0067 - val_loss: 31.4902 - val_mean_squared_error: 0.0052
Epoch 3/5
2688/2688 [==============================] - 31s 11ms/step - loss: 30.5293 - mean_squared_error: 0.0060 - val_loss: 29.6905 - val_mean_squared_error: 0.0056
Epoch 4/5
2688/2688 [==============================] - 31s 11ms/step - loss: 29.6575 - mean_squared_error: 0.0056 - val_loss: 28.7736 - val_mean_squared_error: 0.0050
Epoch 5/5
2688/2688 [==============================] - 31s 11ms/step - loss: 28.7102 - mean_squared_error: 0.0052 - val_loss: 27.8096 - val_mean_squared_error: 0.0050


We also try `Hyperband` which generates a large number of models and discard them in a tournament style.

In [ ]:
band_tuner = keras_tuner.Hyperband(
    hypermodel=tuneLR,
    objective="val_loss",
    max_epochs=10,
    factor=3,
    directory="hyperparams/Hyperband",
    project_name="1"
)
band_tuner.search_space_summary()
band_tuner.search(train_ds, epochs = 2, validation_data = valid_ds)  

In [ ]:
band_tuner.results_summary()

Finally, we try `BayesianOptimization`.

In [ ]:
bayes_tuner = keras_tuner.BayesianOptimization(
    hypermodel=tuneLR,
    objective="val_loss",
    max_trials=3,
    directory="hyperparams/BayesianOptimization",
    project_name="1"
)
bayes_tuner.search_space_summary()
bayes_tuner.search(train_ds, epochs = 2, validation_data = valid_ds) 

In [ ]:
bayes_tuner.results_summary()